# Module 3, Section 2: Interacting with Deployments via SDK

Our agent is now deployed to production with monitoring in place. But how do we actually **use** it?

In this section, we'll learn how to interact with our deployed agent programmatically using the **LangGraph SDK**. This enables you to:
- Build custom UIs and applications
- Automate testing and workflows
- Handle human-in-the-loop interrupts
- Integrate with existing systems

By the end, you'll be able to call your deployed agent from any Python application.


## Setup


In [ ]:
from dotenv import load_dotenv
from langgraph_sdk import get_client
from langgraph_sdk.schema import Command

load_dotenv()

# Connect to your deployment
DEPLOYMENT_API_URL = "your-deployment-api-url"
client = get_client(url=DEPLOYMENT_API_URL)

---

## 1. Basic Invocation

Let's start simple - call our deployed agent and get a response.

We'll ask about product information (no identity verification needed).


**Note:** First invocation to subagent that uses vector search will build the vector DB on-demand, one time setup.

In [2]:
# Create a thread (conversation session)
thread = await client.threads.create()

# Send a message
input_message = {
    "messages": [
        {
            "role": "user",
            "content": "What are the key features of the Sony WH-1000XM5 headphones?",
        }
    ]
}

# Invoke the agent (wait for completion)
response = await client.runs.wait(
    thread["thread_id"],
    "supervisor_hitl_sql_agent",  # Your deployed graph name
    input=input_message,
)

# Print the final response
final_message = response["messages"][-1]["content"]
print(f"Agent: {final_message}")

Agent: Based on our product documentation, here are the **key features of the Sony WH-1000XM5 headphones**:

**Noise Cancellation:**
- Industry-leading Active Noise Cancellation (ANC) with 8 microphones
- HD Noise Cancelling Processor QN1 that adjusts 700 times per second
- Excellent at blocking low-frequency noise (planes, traffic) and mid-frequency sounds (conversations, office noise)
- NC Optimizer feature personalizes cancellation to your ear shape and fit

**Audio Quality:**
- Premium 30mm drivers delivering rich, detailed sound
- Compatible with all Bluetooth devices (iOS, Android, Windows, Mac)

**Battery & Connectivity:**
- **30-hour battery life** (exceptional for wireless headphones)
- Seamless multi-device connectivity

**Microphone & Call Quality:**
- 4 beamforming microphones for clear call quality
- 4 ANC microphones

**Smart Controls:**
- Touch-sensitive controls on the right earcup: tap to play/pause, double-tap to skip, swipe for volume
- Quick Attention mode to tempor

**What just happened?**
1. Created a new thread (like starting a new conversation)
2. Sent a message to the agent
3. Waited for the agent to complete
4. Retrieved the final message

**Note:** Check LangSmith - you'll see this trace in your `langsmith-agent-lifecycle-workshop-deployment` project!


---

## 2. Human-in-the-Loop (HITL) with SDK

Our agent requires email verification for personal account questions. Let's see how to handle **interrupts** programmatically.


### Step 1: Start a Conversation That Requires Verification

**Note:** LangSmith Studio automatically handles interrupt resumption in its UI, but when using the SDK programmatically, you need to explicitly use the `Command(resume=...)` pattern (see Step 3).


In [10]:
# Create a new thread
thread = await client.threads.create()
thread_id = thread["thread_id"]

# Ask about order status (requires email verification)
input_message = {
    "messages": [{"role": "user", "content": "What's the status of my recent order?"}]
}

# Run the graph until the interrupt is hit
result = await client.runs.wait(
    thread_id,
    "supervisor_hitl_sql_agent",
    input=input_message,
)

# Print the interrupt
print(result["__interrupt__"])

[{'id': '23754cfd4d40de50b188c85d3bbd4cee', 'value': 'Please provide your email:'}]


### Step 2: Check the Interrupt and Get State


In [11]:
# Get the current state
state = await client.threads.get_state(thread_id)

# The agent is waiting for email input
# Check the last message
last_message = state["values"]["messages"][-1]["content"]
print(f"Agent says: {last_message}")

Agent says: To access information about your account or orders, please provide your email address.


### Step 3: Provide Email and Resume

**Important:** To resume an interrupted run:
1. Call `client.runs.wait()` again on the **same thread**
2. Pass `command=Command(resume=...)` with the user's input (as a string)
3. The agent resumes from the interrupt point with the provided value

**Key:** The `Command(resume=...)` tells the SDK to resume the interrupted run rather than starting a new one. The value you pass to `resume` becomes the return value of the `interrupt()` call in your graph code.


In [14]:
# Resume the graph by passing the email as a Command
# Note: We pass the user's message content directly to Command(resume=...)
response = await client.runs.wait(
    thread_id,
    "supervisor_hitl_sql_agent",
    command=Command(resume="My email is consultant.tech@gmail.com"),
)

# Get the final answer
final_message = response["messages"][-1]["content"]
print(f"\nAgent: {final_message}")


Agent: Your recent order **ORD-2025-0044** is currently in **Processing** status. Here are the details:

- **Order Date:** October 15, 2025
- **Total Amount:** $549.00
- **Status:** Processing
- **Shipped Date:** Not yet shipped
- **Tracking Number:** Not yet available

Your order is being prepared and will be shipped soon. Once it's dispatched, you'll receive tracking information so you can monitor its delivery progress. Is there anything else you'd like to know about this order?


### Step 4: Ask a Follow-Up (No Interrupt This Time!)


In [15]:
# Ask a follow-up question (customer_id already verified!)
followup_message = {
    "messages": [{"role": "user", "content": "What exactly was in that order?"}]
}

response = await client.runs.wait(
    thread_id,
    "supervisor_hitl_sql_agent",
    input=followup_message,
)

final_message = response["messages"][-1]["content"]
print(f"Agent: {final_message}")

Agent: Your order **ORD-2025-0044** contains:

- **1x Dell UltraSharp 27" 4K Monitor** - $549.00

This is a high-quality 4K monitor, and it's currently in stock. Your order is being prepared for shipment. Is there anything else you'd like to know about your order or this product?


**What happened?**
1. First question triggered email verification (interrupt)
2. We resumed with `Command(resume=...)` to provide the email
3. Follow-up question didn't interrupt - `customer_id` is already in thread state!

**Key HITL Pattern:**
- Use `command=Command(resume=value)` on the same thread to resume an interrupt
- The value becomes the return value of `interrupt()` in your graph
- Thread state (like `customer_id`) persists across runs on the same thread

This is how you build custom UIs with HITL flows.

**Learn more:** [HITL with SDK Docs](https://docs.langchain.com/langsmith/add-human-in-the-loop)


---

## 3. Monitoring Production Traces

Every SDK call creates a trace in LangSmith. Let's verify our online evaluator is working.


### Check LangSmith for Your Traces

**Steps:**
1. Go to **LangSmith** → **Projects** → **langsmith-agent-lifecycle-workshop-deployment**
2. You'll see all the traces from your SDK calls
3. Click on any trace to see:
   - Full conversation history
   - Tool calls and reasoning
   - Latency metrics
   - Any errors or warnings

**After 1 minute** (your thread idle time), check the **Feedback** tab on each trace:
- You should see `user_sentiment: positive` (since we had helpful conversations)
- The online evaluator runs automatically!
